# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 13 2022 8:30AM,248514,10,0086071034,ISDIN Corporation,Released
1,Oct 13 2022 8:30AM,248514,10,0086071033,ISDIN Corporation,Released
2,Oct 13 2022 8:30AM,248514,10,0086071040,ISDIN Corporation,Released
3,Oct 13 2022 8:30AM,248514,10,0086071037,ISDIN Corporation,Released
4,Oct 13 2022 8:30AM,248514,10,0086071038,ISDIN Corporation,Released
5,Oct 13 2022 8:30AM,248514,10,0086071039,ISDIN Corporation,Released
6,Oct 13 2022 8:30AM,248514,10,0086071055,ISDIN Corporation,Released
7,Oct 13 2022 8:30AM,248514,10,0086071056,ISDIN Corporation,Released
8,Oct 13 2022 8:30AM,248514,10,0086071057,ISDIN Corporation,Released
9,Oct 13 2022 8:30AM,248514,10,0086071058,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,248514,Released,33
33,248515,Released,10
34,248516,Released,4
35,248517,Released,2
36,248518,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248514,NaN,NaN,33.0
248515,NaN,NaN,10.0
248516,NaN,NaN,4.0
248517,NaN,NaN,2.0
248518,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248265,28.0,0.0,2.0
248434,0.0,0.0,1.0
248447,0.0,0.0,1.0
248450,0.0,0.0,1.0
248453,0.0,0.0,9.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248265,28,0,2
248434,0,0,1
248447,0,0,1
248450,0,0,1
248453,0,0,9


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248265,28,0,2
1,248434,0,0,1
2,248447,0,0,1
3,248450,0,0,1
4,248453,0,0,9


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248265,28,,2
1,248434,,,1
2,248447,,,1
3,248450,,,1
4,248453,,,9


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 13 2022 8:30AM,248514,10,ISDIN Corporation
33,Oct 13 2022 8:30AM,248515,10,ISDIN Corporation
43,Oct 13 2022 8:30AM,248516,10,ISDIN Corporation
47,Oct 12 2022 4:06PM,248518,20,Alumier Labs Inc.
49,Oct 12 2022 4:04PM,248517,19,ACG North America LLC
51,Oct 12 2022 3:56PM,248512,19,"GCH Granules USA, Inc."
52,Oct 12 2022 3:49PM,248489,15,"Virtus Pharmaceuticals, LLC"
66,Oct 12 2022 3:22PM,248510,10,ISDIN Corporation
80,Oct 12 2022 3:08PM,248507,10,Yusen – 3D Matrix
81,Oct 12 2022 2:45PM,248501,16,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 13 2022 8:30AM,248514,10,ISDIN Corporation,,,33
1,Oct 13 2022 8:30AM,248515,10,ISDIN Corporation,,,10
2,Oct 13 2022 8:30AM,248516,10,ISDIN Corporation,,,4
3,Oct 12 2022 4:06PM,248518,20,Alumier Labs Inc.,,,2
4,Oct 12 2022 4:04PM,248517,19,ACG North America LLC,,,2
5,Oct 12 2022 3:56PM,248512,19,"GCH Granules USA, Inc.",,,1
6,Oct 12 2022 3:49PM,248489,15,"Virtus Pharmaceuticals, LLC",,,14
7,Oct 12 2022 3:22PM,248510,10,ISDIN Corporation,,,14
8,Oct 12 2022 3:08PM,248507,10,Yusen – 3D Matrix,,1,
9,Oct 12 2022 2:45PM,248501,16,ACG North America LLC,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 13 2022 8:30AM,248514,10,ISDIN Corporation,33,,
1,Oct 13 2022 8:30AM,248515,10,ISDIN Corporation,10,,
2,Oct 13 2022 8:30AM,248516,10,ISDIN Corporation,4,,
3,Oct 12 2022 4:06PM,248518,20,Alumier Labs Inc.,2,,
4,Oct 12 2022 4:04PM,248517,19,ACG North America LLC,2,,
5,Oct 12 2022 3:56PM,248512,19,"GCH Granules USA, Inc.",1,,
6,Oct 12 2022 3:49PM,248489,15,"Virtus Pharmaceuticals, LLC",14,,
7,Oct 12 2022 3:22PM,248510,10,ISDIN Corporation,14,,
8,Oct 12 2022 3:08PM,248507,10,Yusen – 3D Matrix,,1,
9,Oct 12 2022 2:45PM,248501,16,ACG North America LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 13 2022 8:30AM,248514,10,ISDIN Corporation,33,,
1,Oct 13 2022 8:30AM,248515,10,ISDIN Corporation,10,,
2,Oct 13 2022 8:30AM,248516,10,ISDIN Corporation,4,,
3,Oct 12 2022 4:06PM,248518,20,Alumier Labs Inc.,2,,
4,Oct 12 2022 4:04PM,248517,19,ACG North America LLC,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 13 2022 8:30AM,248514,10,ISDIN Corporation,33.0,NaN,NaN
1,Oct 13 2022 8:30AM,248515,10,ISDIN Corporation,10.0,NaN,NaN
2,Oct 13 2022 8:30AM,248516,10,ISDIN Corporation,4.0,NaN,NaN
3,Oct 12 2022 4:06PM,248518,20,Alumier Labs Inc.,2.0,NaN,NaN
4,Oct 12 2022 4:04PM,248517,19,ACG North America LLC,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 13 2022 8:30AM,248514,10,ISDIN Corporation,33.0,0.0,0.0
1,Oct 13 2022 8:30AM,248515,10,ISDIN Corporation,10.0,0.0,0.0
2,Oct 13 2022 8:30AM,248516,10,ISDIN Corporation,4.0,0.0,0.0
3,Oct 12 2022 4:06PM,248518,20,Alumier Labs Inc.,2.0,0.0,0.0
4,Oct 12 2022 4:04PM,248517,19,ACG North America LLC,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3230268,88.0,34.0,10.0
15,496754,16.0,0.0,28.0
16,745425,3.0,0.0,0.0
18,248462,1.0,0.0,0.0
19,1739401,12.0,3.0,0.0
20,248518,2.0,0.0,0.0
22,745494,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3230268,88.0,34.0,10.0
1,15,496754,16.0,0.0,28.0
2,16,745425,3.0,0.0,0.0
3,18,248462,1.0,0.0,0.0
4,19,1739401,12.0,3.0,0.0
5,20,248518,2.0,0.0,0.0
6,22,745494,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,88.0,34.0,10.0
1,15,16.0,0.0,28.0
2,16,3.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,12.0,3.0,0.0
5,20,2.0,0.0,0.0
6,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,88.0
1,15,Released,16.0
2,16,Released,3.0
3,18,Released,1.0
4,19,Released,12.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,22
Status,,,,,,,
Completed,10.0,28.0,0.0,0.0,0.0,0.0,0.0
Executing,34.0,0.0,0.0,0.0,3.0,0.0,0.0
Released,88.0,16.0,3.0,1.0,12.0,2.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,22
0,Completed,10.0,28.0,0.0,0.0,0.0,0.0,0.0
1,Executing,34.0,0.0,0.0,0.0,3.0,0.0,0.0
2,Released,88.0,16.0,3.0,1.0,12.0,2.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,22
0,Completed,10.0,28.0,0.0,0.0,0.0,0.0,0.0
1,Executing,34.0,0.0,0.0,0.0,3.0,0.0,0.0
2,Released,88.0,16.0,3.0,1.0,12.0,2.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()